In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

# Load your dataset
df = pd.read_csv('updated_Data_till_T0_2-75_durga_oct17.csv')

# Shuffle the DataFrame
data = df.sample(frac=1, random_state=42)

# Select the first 100 rows for training and testing
data = data[:1_00_000]

# Define input features and target variables
input_features = ['n1', 'l1', 'j1', 'LP', 'Te', 'Z', 'A', 'Rfq', 'T0' ]
target_variables = ['n2', 'l2', 'j2', 'n3', 'l3', 'j3']


# Split the dataset into input (X) and target (y) variables
X = data[input_features]
y = data[target_variables]

# Normalize input features using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define the ANN model as a function
def create_model(learning_rate=0.001, num_units=64, activation='relu'):
    model = models.Sequential([
        layers.Dense(num_units, activation=activation, input_shape=(9,)),
        layers.Dense(num_units, activation=activation),
        layers.Dense(6)  # Output layer with 6 units for n2, l2, j2, n3, l3, j3
    ])
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mean_absolute_error'])
    return model

# Define a function to perform manual hyperparameter tuning
def manual_hyperparameter_tuning(learning_rate_candidates, num_units_candidates, activation_candidates):
    best_mae = float('inf')
    best_hyperparameters = None
    best_history = None

    for lr in learning_rate_candidates:
        for num_units in num_units_candidates:
            for activation in activation_candidates:
                model = create_model(learning_rate=lr, num_units=num_units, activation=activation)
                history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32, verbose=0)
                y_pred = model.predict(X_test)
                mae = mean_absolute_error(y_test, y_pred)

                if mae < best_mae:
                    best_mae = mae
                    best_hyperparameters = {'learning_rate': lr, 'num_units': num_units, 'activation': activation}
                    best_history = history
                    best_model = model  # Store the best model

    return best_hyperparameters, best_history ,best_model ,best_mae

# Define candidate hyperparameter values
learning_rate_candidates = [0.01, 0.1]
num_units_candidates = [32, 64, 128]
activation_candidates = ['relu', 'tanh']


# Perform manual hyperparameter tuning
best_hyperparameters, best_history, best_model,best_mae = manual_hyperparameter_tuning(learning_rate_candidates, num_units_candidates, activation_candidates)

# Get the minimum values
min_training_loss = np.min(best_history.history['loss'])
min_validation_loss = np.min(best_history.history['val_loss'])
min_training_mae = np.min(best_history.history['mean_absolute_error'])
min_validation_mae = np.min(best_history.history['val_mean_absolute_error'])

# Create a DataFrame with hyperparameters and minimum values
data = {
    'Learning Rate': [best_hyperparameters['learning_rate']],
    'Activation': [best_hyperparameters['activation']],
    'Number of Neurons': [best_hyperparameters['num_units']],
    'Minimum Training Loss': [min_training_loss],
    'Minimum Validation Loss': [min_validation_loss],
    'Minimum Training MAE': [min_training_mae],
    'Minimum Validation MAE': [min_validation_mae]
}

df_result = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df_result.to_csv('hyperparameters_and_min_values_1l.csv', index=False)

print("Hyperparameters and minimum values saved to hyperparameters_and_min_values.csv")